In [3]:
import os
import csv
import calendar
import pandas as pd

In [4]:
filename = "./data/Chicago_Crimes_2001_2018.csv"
#filename = "./data/data_small.csv"              # Initial test on small dataset
Chicago_area_mapping_filename = "./data/Chicago_Area_Data.xlsx"
tp = pd.read_csv(filename, low_memory=False, iterator=True, chunksize=10000)  # Read CSV in Chunksize to avoid low memory
data = pd.concat(tp, ignore_index=True)                                       # Concatenate the read dataframes in Chunks
NullSumDF = data.isnull().sum()  # Check is there are Null values in data
NullSumDF

ID                           0
Case Number                  4
Date                         0
Block                        0
IUCR                         0
Primary Type                 0
Description                  0
Location Description      4134
Arrest                       0
Domestic                     0
Beat                         0
District                    47
Ward                    614854
Community Area          616029
FBI Code                     0
X Coordinate             60529
Y Coordinate             60529
Year                         0
Updated On                   0
Latitude                 60529
Longitude                60529
Location                 60529
dtype: int64

In [5]:
# To check percentage of Null data in columns
NullSumDF=data.isnull().sum()
totalRows=len(data)
print("totalRows: ",totalRows)
perDframe = pd.DataFrame((NullSumDF/totalRows)*100, columns=['percent'])
perDframe['colName'] = perDframe.index
print("% of Numm values: ",perDframe)

totalRows:  6752229
% of Numm values:                         percent               colName
ID                    0.000000                    ID
Case Number           0.000059           Case Number
Date                  0.000000                  Date
Block                 0.000000                 Block
IUCR                  0.000000                  IUCR
Primary Type          0.000000          Primary Type
Description           0.000000           Description
Location Description  0.061224  Location Description
Arrest                0.000000                Arrest
Domestic              0.000000              Domestic
Beat                  0.000000                  Beat
District              0.000696              District
Ward                  9.105941                  Ward
Community Area        9.123343        Community Area
FBI Code              0.000000              FBI Code
X Coordinate          0.896430          X Coordinate
Y Coordinate          0.896430          Y Coordinate
Year   

In [6]:
data.dropna(subset=['Case Number', 'Ward','District', 'Latitude', 'Community Area', 'Location Description'], how='any', inplace=True)
data = data.sort_values('Date')  # Sort via Date
os.remove(filename)   # Remove the file. Processed/Cleaned data is in dataframe.
data.to_csv(filename) # Write the data from dataframe back into file.

In [7]:
# Loop through the dataset and add an additional columns like Community name, Month etc.
xl_data = pd.ExcelFile(Chicago_area_mapping_filename)   # Read the excel file. 
                                                        # This file contains the mapping between Community code and Community name.
# print(xl_data.sheet_names[0])
df_area = xl_data.parse(xl_data.sheet_names[0])         # Load the mappig data into the dataframe.
filename_temp = "./data/Chicago_Crimes_2001_2018_temp.csv"   # Temporary file to hold updated data.

header = ['', 'ID', 'Case Number', 'Date', 'Block', 'IUCR', 'Primary Type', 'Description', 'Location Description',
              'Arrest','Domestic', 'Beat', 'District', 'Ward', 'Community Area', 'Community Area Name', 'FBI Code',
              'X Coordinate', 'Y Coordinate', 'Year', 'Month Code', 'Month Name', 'Updated On', 'Latitude', 'Longitude', 
              'Location']

In [8]:
with open(filename_temp, 'w', newline='', encoding='utf8') as tempfile:     # Open file (empty) to write data
      writer = csv.DictWriter(tempfile, fieldnames=header)
      writer.writeheader()
      with open(filename, newline='') as csvfile:                       # Open data file to read data
          print("Generating Data...")
          reader = csv.DictReader(csvfile);
          for row in reader:                                            # Loop through each of the rows in the file
              area_code = int((row['Community Area']).split(".")[0])    # Get Community Code
              area_name = df_area.iloc[area_code, 0]                    # Get the Community name corresponding to Community Code
  
              month_code = (row['Date']).split("/")[0]                  # Get month code
              month_name = calendar.month_name[int(month_code)]         # Get month name
              # Write all the data into the temp file.
              writer.writerow({'': '', 'ID': row['ID'], 'Case Number': row['Case Number'], 'Date': row['Date'],
                               'Block': row['Block'], 'IUCR': row['IUCR'],
                               'Primary Type': row['Primary Type'], 'Description': row['Description'],
                               'Location Description': row['Location Description'], 'Arrest': row['Arrest'],
                               'Domestic': row['Domestic'],
                               'Beat': row['Beat'], 'District': row['District'], 'Ward': row['Ward'],
                               'Community Area': row['Community Area'], 'Community Area Name': area_name,
                               'FBI Code': row['FBI Code'], 'X Coordinate': row['X Coordinate'],
                               'Y Coordinate': row['Y Coordinate'], 'Year': row['Year'], 'Month Name': month_name,
                               'Month Code': int(month_code),
                               'Updated On': row['Updated On'], 'Latitude': row['Latitude'],
                               'Longitude': row['Longitude'], 'Location': row['Location']})
os.remove(filename)
os.rename(filename_temp, filename)  # Rename the temp file to origional file. The temp file contained extra data/columns.
print("Done")
csvfile.close()
tempfile.close()

Generating Data...
Done


In [ ]:
filename = "./data/data_small.csv"
filename_poverty = "./data/Chicago_poverty_and_crime_2004_2013.csv"
df_poverty = pd.read_csv(filename_poverty, header=0)

header = ['', 'ID', 'Case Number', 'Date', 'Block', 'IUCR', 'Primary Type', 'Description', 'Location Description',
              'Arrest','Domestic', 'Beat', 'District', 'Ward', 'Community Area', 'Community Area Name', 
              'Assault (Homicide)', 'Firearm-related', 
              'FBI Code',
              'X Coordinate', 'Y Coordinate', 'Year', 'Month Code', 'Month Name', 'Updated On', 'Latitude', 'Longitude', 
              'Location']

filename_temp = "./data/Chicago_Area_crime_poverty_2004_2013.csv"
with open(filename_temp, 'w', newline='', encoding='utf8') as tempfile:     # Open file (empty) to write data
      writer = csv.DictWriter(tempfile, fieldnames=header)
      writer.writeheader()
      with open(filename, newline='') as csvfile:                       # Open data file to read data
          print("Generating Data...")
          reader = csv.DictReader(csvfile);
          for row in reader:                                            # Loop through each of the rows in the file
              area_code = int((row['Community Area']).split(".")[0])    # Get Community Code
              area_name = df_area.iloc[area_code, 0]                    # Get the Community name corresponding to Community Code
  
              month_code = (row['Date']).split("/")[0]                  # Get month code
              month_name = calendar.month_name[int(month_code)]         # Get month name
              # Write all the data into the temp file.
              writer.writerow({'': '', 'ID': row['ID'], 'Case Number': row['Case Number'], 'Date': row['Date'],
                               'Block': row['Block'], 'IUCR': row['IUCR'],
                               'Primary Type': row['Primary Type'], 'Description': row['Description'],
                               'Location Description': row['Location Description'], 'Arrest': row['Arrest'],
                               'Domestic': row['Domestic'],
                               'Beat': row['Beat'], 'District': row['District'], 'Ward': row['Ward'],
                               'Community Area': row['Community Area'], 'Community Area Name': area_name,
                               'FBI Code': row['FBI Code'], 'X Coordinate': row['X Coordinate'],
                               'Y Coordinate': row['Y Coordinate'], 'Year': row['Year'], 'Month Name': month_name,
                               'Month Code': int(month_code),
                               'Updated On': row['Updated On'], 'Latitude': row['Latitude'],
                               'Longitude': row['Longitude'], 'Location': row['Location']})
os.remove(filename)
os.rename(filename_temp, filename)  # Rename the temp file to origional file. The temp file contained extra data/columns.
print("Done")
csvfile.close()
tempfile.close()